In [1]:
img_size = (512, 1024)


## Data

In [2]:
from PIL import Image

img_path = "doc_2.jpg"
img = Image.open("../dataset/samples/" + img_path)


## ImageProcessor

In [3]:
from typing import Dict
from torchvision.transforms.functional import resize, rotate
from torchvision import transforms
from PIL import ImageOps, Image
import numpy as np
import torch

class ImageProcessor:
    def __init__(self, img_size:tuple = (224,224)):
        self.input_size = img_size
        self.align_long_axis = False
        self.random_padding = False

        self.to_tensor = transforms.Compose(
            [
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
            ]
        )

    def prepare_input(self, img: Image.Image, random_padding: bool = False) -> Dict[str, torch.Tensor]:

        img = img.convert("RGB")
        if self.align_long_axis and (
            (self.input_size[0] > self.input_size[1] and img.height > img.width)
            or (self.input_size[0] < self.input_size[1] and img.height < img.width)
        ):
            img = rotate(img, angle=-90, expand=True)
        img = resize(img, min(self.input_size))
        img.thumbnail((self.input_size[0], self.input_size[1]))
        delta_width = self.input_size[0] - img.width
        delta_height = self.input_size[1] - img.height
        if random_padding:
            pad_width = np.random.randint(low=0, high=delta_width + 1)
            pad_height = np.random.randint(low=0, high=delta_height + 1)
        else:
            pad_width = delta_width // 2
            pad_height = delta_height // 2
        padding = (
            pad_width,
            pad_height,
            delta_width - pad_width,
            delta_height - pad_height,
        )

        pixel_values = torch.Tensor(self.to_tensor(ImageOps.expand(img, padding)))
        return {"pixel_values": pixel_values.unsqueeze(0)}

/Users/johannesscheibe/Library/Caches/pypoetry/virtualenvs/multipage-classifier-DHgYOAvi-py3.10/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
image_processor = ImageProcessor(img_size=img_size)

In [5]:
inputs = image_processor.prepare_input(img)
inputs["pixel_values"].size()

torch.Size([1, 3, 1024, 512])

## SwinEncoder

In [6]:
from transformers import SwinModel, SwinConfig

config = SwinConfig(
    image_size=img_size,
    depths=[2, 2, 14, 2],
    window_size=8,
    patch_size=8,
    embed_dim=128,
    num_heads=[4, 8, 16, 32],
    num_classes=0
)
swin_encoder = SwinModel(config)

In [7]:
swin_encoder.config.image_size

(512, 1024)

In [8]:
with torch.no_grad():
    embeddings = swin_encoder(**inputs)
embeddings.last_hidden_state.size()

torch.Size([1, 128, 1024])